# Prepare data

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import ast
import matplotlib.pyplot as plt
import dill
import torch
import nbimporter
import shap

import os
import sys

os.chdir('/data/repos/actin-personalization/prediction')
sys.path.insert(0, os.path.abspath("src/main/python"))

from models import *
from data.data_processing import DataSplitter, DataPreprocessor
from data.lookups import lookup_manager
from utils.settings import settings
from src.main.python.analysis.predictive_algorithms_training import get_data, plot_different_models_survival_curves

preprocessor = DataPreprocessor(settings.db_config_path, settings.db_name)

In [ ]:
def get_preprocessed_data_with_sourceId(preprocessor):
    df_raw = preprocessor.load_data()
    df_all, updated_features, _ = preprocessor.preprocess_data(
        lookup_manager.features, df=df_raw
    )
    df_all["sourceId"] = df_raw.loc[df_all.index, "sourceId"]
    #df_all["reasonRefrainmentFromTreatment"] = df_raw.loc[df_all.index, "reasonRefrainmentFromTreatment"]
    return df_raw, df_all, updated_features

df_raw, df_all, updated_features = get_preprocessed_data_with_sourceId(preprocessor)

In [ ]:
import json
with open('src/main/python/data/treatment_combinations.json', 'r') as f:
    valid_treatment_combinations = json.load(f)

# Prepare propensity scores

Exclude non covariate columns

In [ ]:
exclude = [
    'hadSurvivalEvent',
    'systemicTreatmentPlan_5-FU',
    'systemicTreatmentPlan_oxaliplatin',
    'systemicTreatmentPlan_irinotecan',
    'systemicTreatmentPlan_bevacizumab',
    'systemicTreatmentPlan_panitumumab',
    'systemicTreatmentPlan_pembrolizumab',
    'systemicTreatmentPlan_nivolumab',
    'hasTreatment',
    'survivalDaysSinceMetastaticDiagnosis',
    'investigatedLymphNodesCountPrimaryDiagnosis',
    'hasRasMutation'
]

df_covariate = df_all.copy()

#df_covariate["hasInvestigatedLymphNodes"] = (
#    df_covariate["investigatedLymphNodesCountPrimaryDiagnosis"] > 0
#)

df_covariate = df_covariate.drop(columns=exclude, errors='ignore')



Create treatment table

In [ ]:
treatment_prefix = "systemicTreatmentPlan_"
treatment_cols = [col for col in df_all.columns if col.startswith(treatment_prefix)]

def extract_actual_treatment(row):
    actual_treatments = [col for col in treatment_cols if row[col] == 1]
    if actual_treatments:
        return ", ".join([col.replace(treatment_prefix, "") for col in actual_treatments])
    else:
        return "No Treatment"

df_all["actual_treatment"] = df_all.apply(extract_actual_treatment, axis=1)


# Train model

Model trained on all data

import pandas as pd
import numpy as np
from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

# 1. Prepare treatments and covariates
treatments = pd.Categorical(df_all["actual_treatment"])  # treat as categorical
treatments_encoded = treatments.codes                             # integer-encoded labels
treatment_labels = treatments.categories                 # original treatment names

covariates = df_covariate.copy()
covariates_encoded = pd.get_dummies(covariates, drop_first=True)

# 2. Drop constant columns (needed before scaling)
constant_cols = covariates_encoded.columns[covariates_encoded.std() == 0]
if len(constant_cols) > 0:
    print(f"Dropping constant columns: {list(constant_cols)}")
    covariates_encoded = covariates_encoded.drop(columns=constant_cols)

# 3. Reset index for alignment
covariates_encoded = covariates_encoded.reset_index(drop=True)
df_all = df_all.reset_index(drop=True)

# 4. Build pipeline with StandardScaler and XGBClassifier
pipe = Pipeline([
    ("scale", StandardScaler()),
    ("xgb", XGBClassifier(
        objective="multi:softprob",           # output class probabilities
        num_class=len(treatment_labels),      # number of unique treatments
        use_label_encoder=False,
        eval_metric="mlogloss",
        max_depth=4,
        n_estimators=200,
        learning_rate=0.1,
        random_state=42
    ))
])



# 5. Fit the model
pipe.fit(covariates_train, treatments_train)

# 6. Predict propensity scores
propensity_probs = pipe.predict_proba(covariates_encoded)

# 7. Add propensity scores to df_all using original treatment labels
for i, label in enumerate(treatment_labels):
    df_all[f"propensity_{label}"] = propensity_probs[:, i]



Model trained on train data (with split test set)

In [ ]:
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split

treatments = pd.Categorical(df_all["actual_treatment"])
treatments_encoded = treatments.codes
treatment_labels = treatments.categories

covariates = df_covariate.copy()
covariates_encoded = pd.get_dummies(covariates, drop_first=True)

constant_cols = covariates_encoded.columns[covariates_encoded.std() == 0]
if len(constant_cols) > 0:
    print(f"Dropping constant columns: {list(constant_cols)}")
    covariates_encoded = covariates_encoded.drop(columns=constant_cols)

covariates_encoded = covariates_encoded.reset_index(drop=True)
df_all = df_all.reset_index(drop=True)

covariates_train, covariates_test, treatments_train, treatments_test = train_test_split(
    covariates_encoded, treatments_encoded, test_size=0.2, random_state=42, stratify=treatments_encoded
)

pipe = Pipeline([
    ("scale", StandardScaler()),
    ("xgb", XGBClassifier(
        objective="multi:softprob",      
        num_class=len(treatment_labels),
        use_label_encoder=False,
        eval_metric="mlogloss",
        max_depth=4,                     
        n_estimators=100,                  
        learning_rate=0.1,                 
        subsample=0.7,                      
        random_state=42,
        verbosity=0
    ))
])

pipe.fit(covariates_train, treatments_train)

propensity_probs = pipe.predict_proba(covariates_encoded)

for i, label in enumerate(treatment_labels):
    df_all[f"propensity_{label}"] = propensity_probs[:, i]


# Random patient generation

In [ ]:
import random
valid_ids = set(df_all['sourceId'].unique())

def get_valid_random_patient_id():
    while True:
        random_id = random.choice(range(1, 10000000))
        if random_id in valid_ids:
            return random_id

random_patient_id = get_valid_random_patient_id()
print(random_patient_id)


import random
threshold_days = 2000 
eligible_patients = df_all[
    (df_all['hasTreatment'] == 0) &
    (df_all['survivalDaysSinceMetastaticDiagnosis'] > threshold_days)
]

valid_ids = set(eligible_patients['sourceId'].unique())

def get_valid_random_patient_id():
    if not valid_ids:
        raise ValueError("No valid patients meet the criteria.")
    return random.choice(list(valid_ids))

random_patient_id = get_valid_random_patient_id()
print(random_patient_id)


# Personalized patient propensity score estimation

In [ ]:
import pandas as pd
import numpy as np
from collections import Counter
import re
from IPython.display import display, Markdown
import shap

patient_id = random_patient_id
row = df_all[df_all["sourceId"] == patient_id]
treatment_row = df_all[df_all["sourceId"] == patient_id].squeeze()
raw_row = df_raw[df_raw["sourceId"] == patient_id].squeeze()

model = pipe.named_steps["xgb"]
scaler = pipe.named_steps["scale"]
feature_names = covariates_encoded.columns
treatment_labels = pd.Categorical(df_all["actual_treatment"]).categories

X_scaled = scaler.transform(covariates_encoded)
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X_scaled) 

patient_idx = df_all.index[df_all["sourceId"] == patient_id][0]
shap_patient = [sv[patient_idx] for sv in shap_values]
base_values = explainer.expected_value
logits = np.array([base_values[i] + shap_patient[i].sum() for i in range(len(base_values))])
probs = np.exp(logits) / np.sum(np.exp(logits))

top_two_idx = np.argsort(probs)[-2:][::-1]
top_idx, second_idx = top_two_idx
top_class = treatment_labels[top_idx]
second_class = treatment_labels[second_idx]
top_contrib = shap_patient[top_idx]
second_contrib = shap_patient[second_idx]

avg_other_contrib = np.mean([shap_patient[i] for i in range(len(treatment_labels)) if i != top_idx], axis=0)
delta_contrib_first = top_contrib - avg_other_contrib
delta_contrib_second = top_contrib - second_contrib
actual_values = row[feature_names].values.flatten()

df_first = pd.DataFrame({
    "feature": feature_names,
    "value": np.round(actual_values, 2),
    "top_contribution": np.round(top_contrib, 2),
    "weighted_avg_other_contribution": np.round(avg_other_contrib, 2),
    "Δ_contribution": np.round(delta_contrib_first, 2)
})
df_first = df_first[df_first["Δ_contribution"] > 0].sort_values("Δ_contribution", ascending=False)

df_second = pd.DataFrame({
    "feature": feature_names,
    "value": np.round(actual_values, 2),
    f"{top_class} contribution": np.round(top_contrib, 2),
    f"{second_class} contribution": np.round(second_contrib, 2),
    "Δ_contribution": np.round(delta_contrib_second, 2)
})
df_second["abs_Δ_contribution"] = df_second["Δ_contribution"].abs()
df_second = df_second.sort_values("abs_Δ_contribution", ascending=False)

def interpret_value(v):
    if v == 1.0: return "positive"
    if v == 0.0: return "negative"
    if v > 0: return "relatively high"
    if v < 0: return "relatively low"
    return "unknown"

def get_raw_value(encoded_feature):
    if encoded_feature in raw_row:
        return raw_row[encoded_feature]
    prefix = encoded_feature.split('_')[0]
    return raw_row.get(prefix, np.nan)

def prettify_feature(feat):
    if "_" in feat and not feat.startswith("has"):
        base, category = feat.split("_", 1)
        base = re.sub(r'(?<!^)(?=[A-Z])', ' ', base).strip().capitalize()
        category = category.replace("_", " ").lower().capitalize()
        return f"{base}: {category}"
    else:
        return re.sub(r'(?<!^)(?=[A-Z])', ' ', feat).strip().capitalize()

for df in [df_first, df_second]:
    df["interpretation"] = df["value"].apply(interpret_value)
    df["actual_value"] = df["feature"].apply(get_raw_value)
    df["feature_written"] = df["feature"].apply(prettify_feature)

propensity_cols = [col for col in df_all.columns if col.startswith("propensity_")]
patient_propensity = row[propensity_cols].T
patient_propensity.columns = ["Propensity Score"]
patient_propensity.index = [col.replace("propensity_", "") for col in patient_propensity.index]
patient_propensity["Propensity Score"] = patient_propensity["Propensity Score"].astype(float).round(2)
patient_propensity = patient_propensity[patient_propensity["Propensity Score"] >= 0.05]
patient_propensity = patient_propensity.sort_values(by="Propensity Score", ascending=False)

treatment_cols = [col for col in treatment_row.index if col.startswith("systemicTreatmentPlan_")]
actual_treatments = [
    col.replace("systemicTreatmentPlan_", "")
    for col in treatment_cols
    if str(treatment_row[col]).strip() in {"1", "1.0", "True", "true"}
]
actual_treatment_str = " + ".join(actual_treatments) if actual_treatments else "No treatment"
survival_days = treatment_row.get("survivalDaysSinceMetastaticDiagnosis", "Unknown")

top_score_percent = f"{patient_propensity.loc[top_class, 'Propensity Score'] * 100:.0f}%" if top_class in patient_propensity.index else "N/A"
if second_class in patient_propensity.index:
    second_score_percent = f"{patient_propensity.loc[second_class, 'Propensity Score'] * 100:.0f}%"
    show_second = True
else:
    second_score_percent = None
    show_second = False

output = [
    f"**Actual Treatment:** {actual_treatment_str}  ",
    f"**Observed Survival:** {survival_days} days",
    ""
]

output.append(f"**{top_class}** ({top_score_percent})")
for _, row_ in df_first[df_first["Δ_contribution"] > 0.1].iterrows():
    output.append(f"- **{row_['feature_written']} →** {row_['interpretation']} ({row_['actual_value']})")

if show_second:
    output.append("")
    output.append(f"**{second_class}** ({second_score_percent})")
    for _, row_ in df_second[df_second["Δ_contribution"] < -0.1].iterrows():
        output.append(f"- **{row_['feature_written']} →** {row_['interpretation']} ({row_['actual_value']})")

output.append("")
for treatment, score in patient_propensity["Propensity Score"].items():
    if treatment not in [top_class, second_class] and score >= 0.05:
        output.append(f"**{treatment}** ({score * 100:.0f}%)")
        output.append("")

display(Markdown("\n".join(output)))


# Accuracy estimation

Based on all data

In [ ]:
import re
from tqdm import tqdm

def normalize_treatment_string(treat_str):
    if not treat_str or treat_str.strip().lower() in {"none", "no treatment"}:
        return set()
    parts = re.split(r"[,+]", treat_str)
    return set(p.strip().lower() for p in parts if p.strip())

model = pipe.named_steps["xgb"]
scaler = pipe.named_steps["scale"]
feature_names = covariates_encoded.columns
treatment_labels = pd.Categorical(df_all["actual_treatment"]).categories

X_scaled = scaler.transform(covariates_encoded)
explainer = shap.TreeExplainer(model)
shap_values_all = explainer.shap_values(X_scaled) 
base_values = explainer.expected_value

results = []

for i, row in tqdm(df_all.iterrows(), total=len(df_all)):
    source_id = row["sourceId"]
    
    treatment_cols = [col for col in row.index if col.startswith("systemicTreatmentPlan_")]
    actual_treatments = [
        col.replace("systemicTreatmentPlan_", "")
        for col in treatment_cols
        if str(row[col]).strip() in {"1", "1.0", "True", "true"}
    ]
    actual_treatment_str = " + ".join(actual_treatments) if actual_treatments else "No treatment"

    shap_patient = [sv[i] for sv in shap_values_all]
    logits = np.array([base_values[j] + shap_patient[j].sum() for j in range(len(base_values))])
    probs = np.exp(logits) / np.sum(np.exp(logits))
    top_idx = np.argmax(probs)
    top_class = treatment_labels[top_idx]
    top_prob = probs[top_idx]

    predicted_set = normalize_treatment_string(top_class)
    actual_set = normalize_treatment_string(actual_treatment_str)
    correct = predicted_set == actual_set

    results.append({
        "sourceId": source_id,
        "predicted_treatment": top_class,
        "predicted_probability": round(top_prob, 3),
        "actual_treatment": actual_treatment_str,
        "correct_match": correct
    })

df_results = pd.DataFrame(results)

accuracy = df_results["correct_match"].mean()
print(f"XGBoost Model Accuracy: {accuracy:.2%} ({df_results['correct_match'].sum()}/{len(df_results)})")

df_mismatches = df_results[~df_results["correct_match"]]
display(df_mismatches.head(10))


Based on test set

In [ ]:
import re
from tqdm import tqdm
import shap
import numpy as np
import pandas as pd

def normalize_treatment_string(treat_str):
    if not treat_str or treat_str.strip().lower() in {"none", "no treatment"}:
        return set()
    parts = re.split(r"[,+]", treat_str)
    return set(p.strip().lower() for p in parts if p.strip())

df_test = df_all[df_all["sourceId"].isin(df_all.loc[covariates_test.index, "sourceId"])].reset_index(drop=True)
X_test = covariates_test.reset_index(drop=True)

model = pipe.named_steps["xgb"]
scaler = pipe.named_steps["scale"]
feature_names = covariates_encoded.columns
treatment_labels = pd.Categorical(df_all["actual_treatment"]).categories

X_scaled_test = scaler.transform(X_test)
explainer = shap.TreeExplainer(model)
shap_values_test = explainer.shap_values(X_scaled_test) 
base_values = explainer.expected_value

results = []

for i, row in tqdm(df_test.iterrows(), total=len(df_test)):
    source_id = row["sourceId"]

    # Actual treatment from original encoding
    treatment_cols = [col for col in row.index if col.startswith("systemicTreatmentPlan_")]
    actual_treatments = [
        col.replace("systemicTreatmentPlan_", "")
        for col in treatment_cols
        if str(row[col]).strip() in {"1", "1.0", "True", "true"}
    ]
    actual_treatment_str = " + ".join(actual_treatments) if actual_treatments else "No treatment"

    shap_patient = [sv[i] for sv in shap_values_test]
    logits = np.array([base_values[j] + shap_patient[j].sum() for j in range(len(base_values))])
    probs = np.exp(logits) / np.sum(np.exp(logits))
    top_idx = np.argmax(probs)
    top_class = treatment_labels[top_idx]
    top_prob = probs[top_idx]

    predicted_set = normalize_treatment_string(top_class)
    actual_set = normalize_treatment_string(actual_treatment_str)
    correct = predicted_set == actual_set

    results.append({
        "sourceId": source_id,
        "predicted_treatment": top_class,
        "predicted_probability": round(top_prob, 3),
        "actual_treatment": actual_treatment_str,
        "correct_match": correct
    })

df_results = pd.DataFrame(results)
accuracy = df_results["correct_match"].mean()
print(f"XGBoost Model Accuracy (Test Set): {accuracy:.2%} ({df_results['correct_match'].sum()}/{len(df_results)})")

df_mismatches = df_results[~df_results["correct_match"]]
display(df_mismatches.head(10))


In [ ]:
print(df_results)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt


df_results['actual_treatment'] = df_results['actual_treatment'].str.replace(" \+ ", ", ").str.title()
df_results['predicted_treatment'] = df_results['predicted_treatment'].str.title()


cross_tab = pd.crosstab(
    df_results['predicted_treatment'],
    df_results['actual_treatment'],
    normalize='index'
) * 100


ax = cross_tab.plot(kind='bar', stacked=True, figsize=(12, 6))
plt.ylabel('Percentage of Actual Treatments')
plt.title('Distribution of Actual Treatments per Predicted Treatment')
plt.legend(title='Actual Treatment', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()


In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint, uniform
from xgboost import XGBClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split


In [ ]:
param_distributions = {
    "xgb__n_estimators": randint(100, 800),
    "xgb__max_depth": randint(3, 10),
    "xgb__learning_rate": uniform(0.01, 0.3),
    "xgb__subsample": uniform(0.5, 0.5),
    "xgb__colsample_bytree": uniform(0.5, 0.5),
    "xgb__gamma": uniform(0, 5),
    "xgb__reg_lambda": uniform(0, 5),
    "xgb__reg_alpha": uniform(0, 5),
}


In [ ]:
pipe = Pipeline([
    ("scale", StandardScaler()),
    ("xgb", XGBClassifier(
        objective="multi:softprob",
        num_class=len(np.unique(treatments_encoded)),
        use_label_encoder=False,
        eval_metric="mlogloss",
        random_state=42,
        verbosity=0
    ))
])

search = RandomizedSearchCV(
    estimator=pipe,
    param_distributions=param_distributions,
    n_iter=50,               
    scoring="neg_log_loss",    
    n_jobs=-1,
    cv=3,                  
    verbose=3,
    random_state=42,
    return_train_score=True
)

search.fit(covariates_train, treatments_train)
